In [37]:
from Grammar import *

FIRST function returns set of terminals that can be deducted from non-terminal that we passed to it.

First(A)
for each rule from non-terminal A:
if first symbol is terminal h => Add to set
else it is a non-terminal B -> Add First(B)

If it is non-terminal B - we get FIRST(B) and add to set, removing empty token

In [38]:
def first_for_all(grammar: Grammar) -> dict:
    result_dict = {}

    def first(var: Var) -> set:
        if var in result_dict.keys():
            # first already calculated
            return result_dict[var]
        else:
            # calculate first
            result = set()
            has_empty = False
            for rule in grammar.variables[var]:
                # index is used in case all rule is a vanishing non-terminals
                for index, symbol in enumerate(rule):
                    if is_empty_token(symbol):
                        has_empty = True

                    if is_token(symbol):
                        result.add(symbol)
                        break  # there is a terminal so not all symbols are vanishing non-terminals
                    else:  # is_var(rule_first)
                        if symbol not in result_dict.keys():
                            result_dict[symbol] = first(symbol)

                        # will not calculate second time because rule_first now in result_dict
                        result = result.union(first(symbol))

                        if get_empty_token() not in result_dict[symbol]:
                            break
                        elif index == len(rule) - 1:
                            # each symbol is a vanish non-terminal => add empty
                            result.add(get_empty_token())

            if not has_empty and result:
                result.discard(get_empty_token())

            return result

    for var in grammar.variables.keys():
        # calculate only if it is not already calculated inside call of FIRST
        if var not in result_dict:
            result_dict[var] = first(var)

    return result_dict


FOLLOW function returns set of terminals that can appear after non-terminal A during deduction

Asserting that end of input is a token (0, '\n')

From the beginning it is true that FOLLOW(main_var) contains (0, '\n')

For each non-terminal, its rule

if S -> (a1)A(a2) then follow(A) += first(a2)
    if first(a2) contains empty token => follow(A) += follow(S).
where (a1) and (a2) are some expressions (list of terminals/non-terminals)

Repeat this process until result is unchanged.

In [39]:
def first_for_sequence(first_dict: dict, seq: Rule):
    result = set()
    for symbol in seq:
        if is_token(symbol):
            result.add(symbol)
            return result
        else:  # is_var(symbol)
            result = set.union(result, first_dict[symbol])
            if get_empty_token() not in first_dict[symbol]:
                return result
            else:
                result.discard(get_empty_token())

    # each symbol has empty token in first so result should also contain empty symbol
    result.add(get_empty_token())
    return result


def follow_for_all(grammar: Grammar) -> dict:
    first_dict = first_for_all(grammar)

    result_dict = {
        non_terminal: set()
        for non_terminal in grammar.variables.keys()
    }
    result_dict[grammar.main_var].add((0, '\n'))

    changed = True
    while changed:
        changed = False
        for non_terminal, definition in grammar.variables.items():
            for rule in definition:
                for index, symbol in enumerate(rule):
                    if is_var(symbol):
                        prev_size = len(result_dict[symbol])

                        first_set = first_for_sequence(first_dict, rule[index + 1:])
                        if get_empty_token() in first_set:
                            first_set.remove(get_empty_token())
                            result_dict[symbol] = set.union(result_dict[symbol], result_dict[non_terminal])

                        result_dict[symbol] = set.union(result_dict[symbol], first_set)

                        new_size = len(result_dict[symbol])
                        if new_size != prev_size:
                            changed = True

    return result_dict

In [40]:
def main():
    grammar = Grammar(
        {(0, ''), (1, 'a'), (1, 'b'), (2, '+'), (2, '-'), (3, 'i')},
        {
            'S' : [ ['A', 'B']                             ],
            'B' : [ [(2, '+'), 'A', 'B'], [(0, '')]        ],
            'A' : [ ['C', 'D']                             ],
            'C' : [ [(1, 'a'), 'S', (1, 'b')], [(3, 'i')]  ],
            'D' : [ [(2, '-'), 'C', 'D'], [(0, '')]        ]
        },
        'S'
    )

    print(first_for_all(grammar))
    print(follow_for_all(grammar))

In [41]:
if __name__ == "__main__":
    main()

{'C': {(3, 'i'), (1, 'a')}, 'A': {(3, 'i'), (1, 'a')}, 'S': {(3, 'i'), (1, 'a')}, 'B': {(2, '+'), (0, '')}, 'D': {(2, '-'), (0, '')}}
{'S': {(0, '\n'), (1, 'b')}, 'B': {(0, '\n'), (1, 'b')}, 'A': {(2, '+'), (0, '\n'), (1, 'b')}, 'C': {(2, '+'), (2, '-'), (0, '\n'), (1, 'b')}, 'D': {(2, '+'), (0, '\n'), (1, 'b')}}
